In [1]:
# https://github.com/youngmihuang/lyrics_application
# https://pypi.org/project/GoogleNews/

from GoogleNews import GoogleNews

import requests 
from bs4 import BeautifulSoup
import pandas as pd

googlenews = GoogleNews()

googlenews.setlang('cn')
googlenews.setperiod('d')
googlenews.setencode('utf-8')
googlenews.clear()


x = input("請輸入要搜尋的關鍵字，將為你搜集相關字詞內容:")
googlenews.search(x)

alldata = googlenews.result()
result = googlenews.gettext()
links = googlenews.get_links()
# print(type(result))
# print(len(result))
# print(alldata)


print()

for n in range(len(result)):
#     print(n+1)
    print(result[n])
    print(links[n])
   

df = pd.DataFrame(
{
    '標題': result,
    '連結': links
})
 
df


請輸入要搜尋的關鍵字，將為你搜集相關字詞內容:台灣

港府突關台灣辦事處外媒：台港關係將更惡化- 國際
https://news.ltn.com.tw/news/world/breakingnews/3537674
台灣人為何每次「智力測驗」都做錯網友揭無奈真相| ETtoday ...
https://www.ettoday.net/news/20210519/1985278.htm
學中國蓋方艙醫院？美專家籲台灣冷靜
https://tw.news.yahoo.com/%E5%AD%B8%E4%B8%AD%E5%9C%8B%E8%93%8B%E6%96%B9%E8%89%99%E9%86%AB%E9%99%A2-%E7%BE%8E%E5%B0%88%E5%AE%B6%E7%B1%B2%E5%8F%B0%E7%81%A3%E5%86%B7%E9%9D%9C-155011516.html
「缺水缺電缺疫苗台灣還能撐多久」
https://tw.news.yahoo.com/%E7%BC%BA%E6%B0%B4%E7%BC%BA%E9%9B%BB%E7%BC%BA%E7%96%AB%E8%8B%97-%E5%8F%B0%E7%81%A3%E9%82%84%E8%83%BD%E6%92%90%E5%A4%9A%E4%B9%85-155621963.html
大陸熱議台灣一定成為「下一個印度」 毫無疑問徹底淪陷 ...
https://www.ettoday.net/news/20210518/1985210.htm
旅外人士最愛國家！台灣連續三年位居榜首- 國際
https://news.ltn.com.tw/news/world/breakingnews/3537487
快訊／今彩539頭獎開出！一注800萬幸運兒在台中| ETtoday ...
https://www.ettoday.net/news/20210518/1985259.htm
梅雨解渴有望吳德榮：這2天滯留鋒南下觸及台灣
https://tw.news.yahoo.com/%E6%A2%85%E9%9B%A8%E8%A7%A3%E6%B8%B4%E6%9C%89%E6%9C%9B-%E5%90%B3%E5%BE%B7%E6%A6%AE-%E9%80%992%E5%A4%A9%E6%BB%AF%E7%

,標題,連結
0,港府突關台灣辦事處外媒：台港關係將更惡化- 國際,https://news.ltn.com.tw/news/world/breakingnew...
1,台灣人為何每次「智力測驗」都做錯網友揭無奈真相| ETtoday ...,https://www.ettoday.net/news/20210519/1985278.htm
2,學中國蓋方艙醫院？美專家籲台灣冷靜,https://tw.news.yahoo.com/%E5%AD%B8%E4%B8%AD%E...
3,「缺水缺電缺疫苗台灣還能撐多久」,https://tw.news.yahoo.com/%E7%BC%BA%E6%B0%B4%E...
4,大陸熱議台灣一定成為「下一個印度」 毫無疑問徹底淪陷 ...,https://www.ettoday.net/news/20210518/1985210.htm
5,旅外人士最愛國家！台灣連續三年位居榜首- 國際,https://news.ltn.com.tw/news/world/breakingnew...
6,快訊／今彩539頭獎開出！一注800萬幸運兒在台中| ETtoday ...,https://www.ettoday.net/news/20210518/1985259.htm
7,梅雨解渴有望吳德榮：這2天滯留鋒南下觸及台灣,https://tw.news.yahoo.com/%E6%A2%85%E9%9B%A8%E...
8,大陸台青呼籲 台灣執政當局應多方獲取疫苗,https://udn.com/news/story/7333/5468088?from=u...
9,暖男！想帶給台灣正面能量宋家豪喊：最高,https://tw.news.yahoo.com/%E6%9A%96%E7%94%B7-%...


In [2]:

url = df['連結'][0]
print(url)


user_agent = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36'}


r = requests.get(url, headers=user_agent)
r.encoding = "utf-8"
web_content = r.text
soup = BeautifulSoup(web_content,'html')


articleContent = soup.find_all('p')
# articleContent

article = []
for p in articleContent:
    article.append(p.text)


articleAll = '\n'.join(article)
# print(articleAll)

# 分段用

https://news.ltn.com.tw/news/world/breakingnews/3537674


In [3]:

import jieba
jieba.load_userdict('dict.txt.big.txt')

d = articleAll.replace('[^\w\s]','').replace('／',"").replace('《','').replace('》','').replace('，','').replace('。','').replace('「','').replace('」','').replace('（','').replace('）','').replace('！','').replace('？','').replace('、','').replace('▲','').replace('…','').replace('：','')
# print(d)

jieba.setLogLevel(20)

# Sentence = jieba.cut(d, cut_all=True)
# print('全模式'+": "  + "/ ".join(Sentence) + '\n')   
 
# Sentence = jieba.cut(d, cut_all=False)
# print('精確模式'+": " + "/ ".join(Sentence)+ '\n')  
 
# Sentence = jieba.cut(d)  
# print('Default為精確模式'+": " + "/ ".join(Sentence)+ '\n')
 
Sentence = jieba.cut_for_search(d)  
print('搜索引擎模式'+": " + "/ ".join(Sentence)+ '\n')

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/2p/qz_90rqs1fx0_vgnt2xgf6yw0000gn/T/jieba.cache
Loading model cost 0.837 seconds.
Prefix dict has been built successfully.


搜索引擎模式: 為/ 達/ 最佳/ 瀏覽/ 效果/ 建議/ 使用/  / ChromeFirefox/  / 或/  / Microsoft/  / Edge/  / 的/ 瀏覽器/ 
/ 
/ 爆/ 
/ 去年/ 6/ 月/ 30/ 日/ 香港/ 推行/ 國家/ 安全/ 安全法/ 後/ 台灣/ 成立/ 了/ 台港/ 服務/ 交流/ 辦/ 公室/ 為/ 逃離/ 香港/ 來/ 到/ 台灣/ 的/ 人/ 提供/ 諮/ 詢服務/ 和/ 人道/ 援助/ 這/ 一行/ 動/ 被/ 北京/ 政府/ 北京政府/ 視為/ 破壞/ 國家統/ 一資料/ 照/ 記者/ 陳/ 志曲攝/ 
/ 〔/ 即時/ 新聞/ 綜合/ 報導/ 〕/ 香港/ 政府/ 香港政府/ 今天/ 突然/ 宣布/ 關閉/ 香港/ 在/ 台灣/ 的/ 經濟/ 貿易/ 文化/ 辦事處/ 外媒/ 對/ 此/ 表示/ 自從/ 北京/ 在/ 香港/ 強推國家/ 安全/ 安全法/ 以來/ 台港/ 關/ 係/ 就/ 不斷/ 惡化/ 
/ 綜合外/ 媒報導/ 香港/ 特別/ 行政/ 區/ 政府/ 今日/ 公告/ 香港/ 經濟/ 貿易/ 文化/ 辦事處/ 台灣/ 自/ 即日/ 5/ 月/ 18/ 日起/ 暫時/ 停止/ 運作/ 香港/ 政府/ 香港政府/ 發言人/ 表示/ 此/ 決定/ 與/ 台灣/ 的/ 疫情/ 狀況/ 無關/ 但/ 沒/ 有給/ 出具/ 體關閉/ 原因/ 
/ 請/ 繼續/ 往下/ 閱讀/ .../ 
/ 
/ 據美國/ 之音/  / 報導/ 去年/ 6/ 月/ 30/ 日/ 香港/ 推行/ 國家/ 安全/ 安全法/ 後/ 台灣/ 成立/ 了/ 台港/ 服務/ 交流/ 辦/ 公室/ 為/ 逃離/ 香港/ 來/ 到/ 台灣/ 的/ 人/ 提供/ 諮/ 詢服務/ 和/ 人道/ 援助/ 這/ 一行/ 動/ 被/ 北京/ 政府/ 北京政府/ 視為/ 破壞/ 國家統/ 一/ 
/ 去年/ 7/ 月台/ 灣駐/ 港/ 辦事處/ 代理/ 處長/ 高銘村/ 因為/ 拒絕/ 簽名/ 保證/ 遵守/ 一/ 中原/ 則/ 因此/ 未能/ 獲得/ 簽證續簽/ 不得/ 不得不/ 離開/ 香港/ 之/ 後/ 7/ 月/ 21/ 日台灣/ 也/ 拒絕/ 了/ 兩名/ 香港/ 官員/ 的/ 簽證/ 申請/ 
/ 接者/ 星期/ 上星期/ 陸委會/

In [4]:
import numpy as np

from PIL import Image

from collections import Counter

import matplotlib.pyplot as plt

from wordcloud import WordCloud, ImageColorGenerator

from scipy.ndimage import gaussian_gradient_magnitude

with open('stopword.txt', 'r', encoding="utf-8") as f:
    stopwords = f.read().split('\n')

terms = {}
for sentence in Sentence:
    if sentence in stopwords:
        continue

    if sentence in terms:
        terms[sentence] += 1
    else:
        terms[sentence] = 1

print(Counter(terms))
# generate_from_text()方法會採納stopwords參數 也可使用

# https://coolors.co/palettes/popular


artDf = pd.DataFrame.from_dict(terms, orient='index', columns=['詞頻'])
artDf.sort_values(by=['詞頻'], ascending=False)

img = "color"
img_path = "%s.png" % img

mask_color = np.array(Image.open(img_path))
mask_color = mask_color[::3, ::3]
mask_image = mask_color.copy()
mask_image[mask_image.sum(axis=2) == 0] = 255

edges = np.mean([gaussian_gradient_magnitude(mask_color[:, :, i] / 255., 2) for i in range(3)], axis=0)
mask_image[edges > .08] = 255

wc = WordCloud(font_path="/Users/larry/Library/Fonts/SourceHanSansTC-Bold.otf",
               mask=mask_color,
               max_font_size=45,
               max_words=2000,
               stopwords=stopwords,
               margin=1,
               relative_scaling=0)

wc.generate_from_frequencies(terms)
image_colors = ImageColorGenerator(mask_color)
wc.recolor(color_func=image_colors)

# 視覺化

plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.figure(figsize=(25, 25))
plt.show()

# plt.savefig("Wordcloud.png")
wc.to_file("img/2021-0518.png")

Counter()


ValueError: We need at least 1 word to plot a word cloud, got 0.

In [5]:

newsDf = pd.DataFrame.from_dict(terms, orient='index', columns = ['詞頻'])
newsDf.sort_values(by= ['詞頻'], ascending= False)

,詞頻
